# Tutorial 07 - Frame Extraction and Gaze Visualization

In this tutorial we will look at how to extract individual frame images from the world video using `ffmpeg`, and visualize gaze positions for a specific frame.

1. Extract all frame images from the world video
1. Load exported gaze positions using `Pandas`
1. Visualize image and gaze positions for a frame index

---
> To execute this notebook, download the [sample recording](https://drive.google.com/file/d/1vzjZkjoi8kESw8lBnsa_k_8hXPf3fMMC/view?usp=sharing). Unzip and move it into the `recordings` directory for this repository.

In [ ]:
from pathlib import Path

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
rec_dir = Path().joinpath("recordings", "sample_recording_v2").resolve()
assert rec_dir.is_dir(), f"Please download the sample_recording_v2 into recordings/"
rec_dir

### Extracting Frames

Most video files generated by Pupil will have a variable frame rate, mirroring the exact frame rate of the incoming frames from the cameras.
A common postprocessing step is extracting every frame from the videos.
You can use a tool like [FFmpeg](https://ffmpeg.org/) for this, but need to keep in mind the variable frame rate.
E.g. for extracting every frame from an exported world video, you can use:
```
ffmpeg -i "/path/to/input/recording/world.mp4" -vsync 0 "/path/to/output/images/frame%06d.png"
```
where the `-vsync 0` option ensures correct handling of the variable frame rate.

In [ ]:
# Delete previously extracted frames
!rm -f ./data/extracted_frames/*.png

In [ ]:
# Extract individual frames from the world video
!ffmpeg -i "./recordings/sample_recording_v2/world.mp4" -vsync 0 "./data/extracted_frames/frame%06d.png"

### Loading Gaze Data

The sample recording contains an export with `gaze_positions.csv` file; we're using `Pandas` to read the data into a data frame for easy access.

In [ ]:
gaze = pd.read_csv(
    rec_dir.joinpath("exports", "000", "gaze_positions.csv")
)
gaze.head(5)

### Visualizing Image and Gaze

In [ ]:
FRAME_INDEX = 1601  # Frame index used for visualization

We read the data of the extracted frame image with the given index.

In [ ]:
# Get the extracted frame image path for the given index
frame_index_path = Path().resolve()
# Append path to extracted frame directory
frame_index_path = frame_index_path.joinpath("data", "extracted_frames")
# File name matching the "frame%06d.png" template
frame_index_path = frame_index_path.joinpath(f"frame{str(FRAME_INDEX).rjust(6, '0')}.png")
assert frame_index_path.is_file(), f"Can't find frame image at path: {frame_index_path}"

# Note that matplotlib's origin is by default in the top-left,
# but Pupil's data is in the bottom-left, so we flip the image and
# use a different origin when calling `imshow()`
frame_index_image = plt.imread(frame_index_path)
frame_index_image = np.flipud(frame_index_image)

frame_index_image.shape

Now we extract the gaze points for the give frame index into an array of `(x, y)` normalized coordinates.

In [ ]:
# Get the array of normalized gaze points for the given index
gaze_points = gaze[gaze["world_index"] == FRAME_INDEX]
gaze_points = gaze_points.sort_values(by="gaze_timestamp")
gaze_points = gaze_points[["norm_pos_x", "norm_pos_y"]]
gaze_points = gaze_points.to_numpy()
gaze_points

Next, we split the gaze points data into the `x` and `y` component arrays, and denormalize it to the size of the frame image.

In [ ]:
# Split gaze points into separate X and Y coordinate arrays
X, Y = gaze_points[:, 0], gaze_points[:, 1]

# Denormalize gaze points within the frame
H, W = frame_index_image.shape[:-1]
X, Y = X * W, Y * H

X, Y

Finally, it's time to plot the image and overlay the gaze data.

In [ ]:
# Plotting configuration
plt.figure(figsize=(16,9))
plt.title(f"Frame #{FRAME_INDEX}")
plt.axis("off")

# Draw the frame image
plt.imshow(frame_index_image, origin="lower")

# Draw the gaze points for the given frame
plt.scatter(X, Y, color=(0.0, 0.7, 0.25), zorder=1, s=700, alpha=0.2)

# Draw the gaze movement line for the given frame
plt.plot(X, Y, color=(1.0, 0.0, 0.4), zorder=2, lw=3)

plt.show()